<img src="logos/Logo_main.png" width=500></img>
# Getting Started

The Purpose of this game day is to learn how to quickly develop and deploy machine learning models in a competitive environment. We will do this by developing models to play the game battleship!

## Game Description and Rules
Your team will create ML models to play "AI Command" which is like battleships! (aka you sunk my battleship) Each team will have a board they place their ships on and will try to guess the placement of the opposing team's ships. During each round of guess the teams will pick a location on the opposing teams board which will be revealed as either a hit or miss. The first team to find all of the opposing teams ships wins!

In order to play the game each team will need to create two ML models: a layout model that places ships on the board and a shoot model the guesses locations of the other teams ships. 

### Rules
- __Board__: Each team in each game will place their ships on a rectangular game board and then guess/shoot the opposing ships board. Board size will given to you. 
- __Layout__:
    - Ships: Ships can be any shape as long as they follow the constraints given for the scenario. The details of these constraints will be in the Layout Endpoint Instructions
- __shoot__: Teams will take one shot at a time until one of the following happens:
    1. one team sinks all the ships of the opposing team
    2. The max number of shooting rounds is reached. The Max number of shooting rounds is equal to the area of the game board. So if a Game Board is 10x10 then their are max 100 shots per team per game
- __Scoring__: team scores will be tracked along three dimensions
    1. Games Won
    1. Total number of hits against opponents 
    1. Total number of hits by opponents

### Architecture

<img src="diagrams/architecture.png"></img>

The engine for game works like this:
1. a cloudwatch alarm triggers the __launcher lambda__ every 30s (or some other time interval)
2. the a launcher lambda reads from the __team dynamodb table__ (which is written to when you register your team) and writes to the Game sqs a game config for each pair of teams. 
3. The __Game lambda__ reads objects from the game queue and runs each game by
    1. calling each teams __layout endpoint__
    2. repeatedly calling each teams __shoot endpoint__ and updating score
    3. when the game is finished it writes the game data to the __game dynamodb table__
    4. if there are any errors from the teams it writes the error to the team's __error sns topic__
4. The __game DynamoDb__ table streams to the stream lambda which:
    1. writes the game data a __kinesis firehose__ which writes the data to an __S3 bucket__ (for the teams to consume and download) and an __elasticsearch database__ (for the game admins to debug).
        1. when new data is written to the s3 bucket a __SNS topic__ is trigger to notify teams of new data
    2. writes the score updates to a __redis cluster__

## Set Up
Your teams success will depend on:

- Ability to work together 
- Ability to deploy models quickly
- Use time effectively
- ML knowledge



###  Organize your Team
To help your team work together and deploy models quickly we recommend to you organize your team into three groups: Opperations, Defense, and Offense. Each group will have their own skills and responsibilities and will work with the other groups to win the game. 

1. __Opperations__: Member with good programing skills but week ML or Data science skill will work well in this group. they are Responsible for: 
    1. Managing/Deploying the Infastructure 
    2. Deploying models
    3. communicating errors and feedback to Layout and Shoot groups
1. __Defensive Group__: Responsible for researching and developing the Layout endpoint. Members who have strong data science skills but not Machine Learning or Deel Learning skills would work will in this group.
1. __Offensive Group__: Responsible for researching and developing the Shooting endpoint. Members with strong machine learning skills will do will in this team.

You also need to give your team a name! 


### Schedule
To help your team use your time effectively we suggest the following schedule for your team and each group. We also recommend you work in "small batch sizes". Try to do many small changes and deployments quickly instead of doing a few large deployments.

<img src="diagrams/schedue.png" width="800"></img>


### Set Up Your Account and Environments
You will need a single AWS Account for your team to play the game and should set it up by following these instructions:

1. Create IAM users for each team member and assign them admin previleges. [Instructions](https://docs.aws.amazon.com/IAM/latest/UserGuide/id_users_create.html)

Each group will work in a SageMaker notebook instance. Use what ever your group feels more comfortable with


## Set up your code 
You team should use git inorder to collaborate on and share your code. To do this you will create a private git repository in the AWS account to hold all your code and then will use the code of this repository to seed or bootstrap the repository with already working code and scripts for you to use.
<img src="diagrams/git-01.png" width="500"></img>

1. Create an AWS CodeCommit Repository with the name __ml-gameday__ (the exact name is important). [instructions](https://docs.aws.amazon.com/codecommit/latest/userguide/how-to-create-repository.html)


## Break into your groups!
break into your three groups and work through the coresponding tutorials

- [Opperations Notebook](/notebooks/Opperations.ipynb)
- [Offensive Notebook](/notebooks/Shoot.ipynb)
- [Defensive Notebook](/notebooks/Layout.ipynb)
